In [1]:
import numpy as np

import tb_hamiltonian.continuum.new as TBHam

model = TBHam.BLGContinuumModel(
        nearest_neighbor_hopping=2.7,
        bond_length=1.42,
        interlayer_hopping=0.27,
        superlattice_potential_periodicity=500,
        superlattice_potential_amplitude=20e-3,
        gate_bias=24e-3,
        layer_potential_ratio=0.3,
        concentric_order=3,
)

In [2]:
# High symmetry points
Γ = np.array([0.0, 0.0])
M = model.b1 / 2
K = (model.b1 - model.b2) / 3

In [4]:
K_new = model.decompose_and_keep_remainder(model.KG)

high_sym_points_K = {
    "Γ": Γ - K_new,
    "M": M - K_new,
    "K": K - K_new,
}

path = "Γ K M Γ"
path_points_K = [high_sym_points_K[k] for k in path.split()]
kpath_K, k_point_indices = TBHam.interpolate_path(path_points_K, total_points=200)

In [5]:
eigenvalues_K, _ = TBHam.compute_eigenstuff(model.H2D_KG, kpath_K)

In [6]:
high_sym_points_Kp = {
    "Γ": Γ + K_new,
    "M": M + K_new,
    "K": K + K_new,
}

# Define paths and interpolate
path_points_Kp = [high_sym_points_Kp[k] for k in path.split()]

kpath_Kp, _ = TBHam.interpolate_path(path_points_Kp, total_points=200)

In [7]:
eigenvalues_Kp, _ = TBHam.compute_eigenstuff(model.H2D_KpG, kpath_Kp)

In [8]:
# Combine and sort eigenvalues
eigenvalues_combined = np.hstack((eigenvalues_K, eigenvalues_Kp))
eigenvalues_sorted = np.sort(eigenvalues_combined, axis=1)

In [9]:
nbands = eigenvalues_K.shape[1]
mid_band = int(nbands / 2)
EF = (np.min(eigenvalues_K[:, mid_band]) + np.max(eigenvalues_K[:, mid_band - 1])) / 2

In [10]:
from tb_hamiltonian.utils import BandStructure

bs = BandStructure(
    high_sym_points=high_sym_points_K,
    path=path,
    distances=[*range(len(eigenvalues_K))],
    eigenvalues=eigenvalues_sorted,
    high_sym_indices=k_point_indices,
    e_fermi=EF,
)

In [ ]:
bs.plot(
    fig_params={
        "figsize": (6, 6),
        "ylim": (-0.045, 0.045),
    },
    plot_params={"color": "black"},
)

In [ ]:
import matplotlib.pyplot as plt

# Initial plot with all bands in black
fig_params = {
    "figsize": (6, 6),
    "ylim": (-0.045, 0.045),
}
plot_params = {"color": "black"}

# Plot all bands initially in black
ax = bs.plot(
    fig_params=fig_params,
    plot_params=plot_params,
)

# Re-plot the mid band on top of the existing plot in red
ax.plot(bs.distances, bs.eigenvalues[:, 2*mid_band], color="red")

plt.show()
